# `Selenium Webscraping Indeed Job Postings - July 2023`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# `Purpose & Outcome:`

# `What is Selenium and how is it used?`

+ When you need to do unit testing, automation or assistance when webscraping this is a tool to aid you.
    + Great for clicking buttons
    + drop-down menus
    + acting/emulating human interactions on a webpage

In [1]:
# Install if you have never used these: unblock the lines below to install if needed

# !pip install webdriver-manager
# !pip3 install lxml
# !pip3 install selenium
# !pip3 install webdriver_manager
# !pip install --upgrade pip
# !pip install -U selenium

In [2]:
# --------- import necessary modules -------

# For webscraping
from bs4 import BeautifulSoup

# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

In [3]:
import selenium

# Check version I am running
selenium.__version__

'4.10.0'

In [4]:
# selenium 4:

from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

# `Consider Headless Browser: speed up & use less resources:`

There are some condiserations though:

+ Some browsers create issues
+ debugging can be tricky
+ you may have limited plugin usage or support
+ you are not able to see visually how the website or application are working 

# `from selenium.webdriver.common.by import By`

Think of this as being similar to using `Beautiful Soup and find_all`
+ when used it allows you to find something within an HTML document, if it fails you raise the exception: `NoSuchElementException`
+ **`Becareful when using BY`** because if this is not a static page then any attrubutes you are searching can become an error in the future when it fails.
    + For example if you are searching by `Class` this can create issues later vs using
        + This is because it is a `CSS` selector and can change overtime since it is an attribute
    + `ID` which may make your code more robust! This CAN be a unique identifier that may help you instead

# `NoSuchElementException`

This is useful to locate elements within a page while loading and try to handle exceptions.
+ During `AJAX` calls you may have issues if the application was build using `React, VUE, Angular` and require different use cases to make the above checks. [article to explain](https://reflect.run/articles/everything-you-need-to-know-about-nosuchelementexception-in-selenium/) and you can consider polling.

`--------------------------------`

# `Other Common Errors:`

+ **`InvalidSelectorException`**

+ **`ElementNotInteractableException`**

+ **`TimeoutException`**

In [6]:
option= webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

# option.add_argument('--headless=chrome')


In [124]:
# Define job and location search keywords
job_search_keyword = ['Data+Scientist', 'Business+Analyst', 'Data+Engineer', 
                      'Python+Developer', 'Full+Stack+Developer', 'Machine+Learning+Engineer']

# Define Locations of Interest
location_search_keyword = ['New+York', 'California', 'Washington']

# Define base and pagination URL's
base_url = 'https://www.indeed.com'

# Finding location, position, radius=35 miles, sort by date and starting page
paginaton_url = 'https://www.indeed.com/jobs?q={}&l={}&radius=35&sort=date&start={}'

print(paginaton_url)

https://www.indeed.com/jobs?q={}&l={}&radius=35&sort=date&start={}


# Things to consider:

+ Wait for page to load before we start running tasks
+ make sure what we are looking for is actually there
    + It can be absent
    + hidden in DOM, iframe or similar
+ timing our calls to remain more like an average user
+ Exception handling

In [162]:
# pagination: PRACTICE

job_='Data+Engineer'
location= 'Washington'
page=[0,10,20]



job_lst=[]

# job_company_list = []
# job_location_list = []
# job_salary_list = []
# job_type_list = []
# job_date_list = []
job_description_list = []
salary_list=[]
# job_link_list = []
# job_id_list = []


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)
sleep(randint(2, 6))

# driver.get("https://www.indeed.com/q-USA-jobs.html")

for i in page:
    driver.get(paginaton_url.format(job_,location,i))
    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
        print(job_title.text)
        
# Href's to get full job description (need to re-terate to get full info)
# Reference ID for each job used by indeed         
# Finding the company name        
# Location
# posting date

        job_lst.append([job_title.text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"),
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"),      
        jj.find_element(By.CLASS_NAME,"companyName").text,       
        jj.find_element(By.CLASS_NAME,"companyLocation").text,
        jj.find_element(By.CLASS_NAME,"date").text])
    
    
#         print(jj.find_element(By.CLASS_NAME,"companyName").text)
        
        # Job Description Links for further parsing and scraping text
        job_description_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"))
        
        # Find Salary if available:
#          if i.find('div',{'class':'attribute_snippet'}) != None:
#                 salary=i.find('div',{'class':'attribute_snippet'}).text

#             else:
#                 salary='No Salary'
        try: 
            salary_list.append(jj.find_element(By.CLASS_NAME,"metadata salary-snippet-container"))
            
#             job_salary_list.append(jj.find(By.XPATH,"/html/body/main/div/div[1]/div/div/div[5]/div[1]/div[4]/div/ul/li[7]/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[3]/div[1]").text)
# #             print(jj.find_element(By.CLASS_NAME,"metadata salary-snippet-container").text)
        except: 
            try: 
                salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
            except:
                salary_list.append(None)







        # Href's to get full job description (need to re-terate to get full info)
#         job_lst.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"))

        # Reference ID for each job used by indeed
#         job_lst.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"))

        # Finding the company name and location
#         job_lst.append(jj.find_element(By.CLASS_NAME,"companyName").text)

#         print(jj.find_element(By.CLASS_NAME,"companyName").text)

        
        # Job location 
#         job_lst.append(jj.find_element(By.CLASS_NAME,"companyLocation").text)

        # Finding posting date
#         job_lst.append(jj.find_element(By.CLASS_NAME,"date").text)

# salary_list  

Data Engineer
Staff Software Engineer - Data Integration
Staff Software Engineer - Data Science
Principal Software Engineer (Data), Industry Solutions Engineering
Data Engineer, AET Central Services Technology - Data
Software Engineer, Data Platform
Customer Engineer, Data Analytics, Google Cloud
Software Engineer, Data Platform
•P-8 Configuration and Data Management/Technical Integration Engineer
Senior Data Engineer
Senior Backend Engineer - Data
Data Center Operations Engineer (Kirkland, WA)
Senior Software Engineer, TikTok Protected Data Infrastructure
Distributed Data Systems - Staff Software Engineer
Solutions Engineer - Data & Machine Learning
Tech Lead Software Engineer, TikTok Protected Data Infrastructure
Staff Full Stack Engineer - Data
Senior Software Engineer - Distributed Data Systems
Lead Software Engineer; Cloud and Big Data
Senior Data Engineer, Kuiper Business Development
Principal Software Engineer, Data (Starshield)
mgr engineer - Data Engineering
Senior Data Engine

In [128]:
# Get links and take description:

# Salary if available and use exceptions

# remote or not! parse with regex and use a separate column for df

#setup for number of jobs and parse by that as iterator

In [ ]:
# convert dates using date time

In [164]:
job_lst[22]
# salary_list
salary_list
job_salary_list

['$140,000 - $190,000 a year',
 '$149,240 - $200,200 a year',
 '$149,240 - $200,200 a year',
 '$133,600 - $256,800 a year',
 'From $105,700 a year',
 '$45 - $51 an hour',
 '$187,000 - $190,300 a year',
 '$140,000 - $215,000 a year',
 '$115,000 - $150,000 a year',
 '$199,500 - $340,100 a year',
 '$182,400 - $247,000 a year',
 '$175,750 - $266,000 a year',
 '$175,000 - $225,000 a year',
 '$157,700 - $213,800 a year',
 '$137,750 - $200,000 a year',
 'From $123,700 a year',
 '$200,000 - $270,000 a year',
 '$177,688 - $341,734 a year',
 '$125,000 - $155,000 a year',
 '$129,960 - $246,240 a year',
 '$115,300 - $172,800 a year',
 'From $109,200 a year',
 '$125,400 - $183,920 a year',
 '$100,531 - $138,230 a year',
 '$95,000 - $130,000 a year',
 '$120,000 - $145,000 a year',
 '$165,000 - $232,000 a year',
 '$143,400 - $175,300 a year',
 '$112,000 - $215,000 a year',
 'From $104,300 a year',
 '$132,900 a year',
 '$125,612 a year',
 '$75 - $85 an hour',
 '$70 - $80 an hour',
 '$173,909 - $198,74

In [130]:
# df and store data

In [131]:
# plots

In [ ]:
# consider NLP

# `Here is a side note:`


+ This gives me an error because it was code from the past version:

`driver = webdriver.Chrome(ChromeDriverManager().install())`


# First let's try to find the number of jobs for a given posting


In [83]:
# Initialize Chrome Webdriver using Chrome Driver Manager
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
#                          options=option)

# sleep(randint(2, 6))
# # driver.implicitly_wait(.72)

# # Open initial URL, and willl wait till its fully loaded.



# # https://www.indeed.com/q-USA-jobs (may suffice if the last ID changes)
# driver.get("https://www.indeed.com/q-USA-jobs.html?vjk=823cd7ee3c203ac3&start=20")
# # sleep(randint(3, 7))

# job_page = driver.find_element(By.ID,"mosaic-jobResults")
# jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list


# job_title_list = []
# job_company_list = []
# job_location_list = []
# job_salary_list = []
# job_type_list = []
# job_date_list = []
# job_description_list = []
# job_link_list = []
# job_id_list = []


# for ii in jobs: 
#     # Finding the job title and its related elements
#     job_title = ii.find_element(By.CLASS_NAME,"jobTitle")
#     print(job_title.text)

#     job_title_list.append(job_title.text)

#     # Href's to get full job description (need to re-terate to get full info)
#     job_link_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"))

#     # Reference ID for each job used by indeed
#     job_id_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"))

#     # Finding the company name and location
#     job_company_list.append(ii.find_element(By.CLASS_NAME,"companyName").text)

#     # Job location 
#     job_location_list.append(ii.find_element(By.CLASS_NAME,"companyLocation").text)

#     # Finding posting date
#     job_date_list.append(ii.find_element(By.CLASS_NAME,"date").text)



Road Production Assistant
Interpreter 1 ( Dari & Pashto)
Client Service Professional- U.S.A. Independent Contractor
Professional Services Consultant (USA)
Housing Program Specialist
Customer Service Advisor and Technical Support Advisor - Mobile Devices and Services - Work-at-Home
Remote Pharmaceutical Contact Center Representative
Elementary Tutor - Math
Coral Restoration Fellow, US Virgin Islands
Support Mentor
Social Work Associate
Field Machinist
Freelance Interviewer
Mechanic
(USA) Member Specialist


In [57]:
# for j in driver.find_elements(By.XPATH,'//*[@id="jobsearch-JapanPage"]/div/div/div[5]/div[1]/nav/div[2]'):
#     print(j)
# driver.fi.
# driver.

In [76]:
job_title_list
job_link_list
# job_id_list
# job_company_list
# job_location_list
job_date_list

# driver.find_element(By.XPATH,'//*[@id="jobsearch-JapanPage"]/div/div/div[5]/div[1]/nav/div[6]').click()

['Employer\nActive 2 days ago',
 'Employer\nActive 10 days ago',
 'Posted\nPosted 19 days ago',
 'Employer\nActive 4 days ago',
 'Employer\nActive 13 days ago',
 'Posted\nPosted 6 days ago',
 'Posted\nPosted 30+ days ago',
 'Posted\nPosted 6 days ago',
 'Employer\nActive 3 days ago',
 'Employer\nActive 2 days ago',
 'Employer\nActive 2 days ago',
 'Posted\nPosted 2 days ago',
 'Posted\nPosted 18 days ago',
 'Employer\nActive 2 days ago',
 'Posted\nToday']

In [ ]:
num_jobs_scraped = 0
job_title_list=[]
# while num_jobs_scraped < 1000:

# The job browing is started
job_page = driver.find_element(By.ID,"mosaic-jobResults")

sleep(randint(3, 8))
# driver.find_elements(By.CLASS_NAME,"pagination")

jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
num_jobs_scraped = num_jobs_scraped + len(jobs)


for ii in jobs: 
    # Finding the job title and its related elements
    job_title = ii.find_element(By.CLASS_NAME,"jobTitle")
    job_title_list.append(job_title.text)
job_title_list

# Add click button after scraping each page

#

In [ ]:
job_title_list = []
# job_company_list = []
# job_location_list = []
# job_salary_list = []
# job_type_list = []
# job_date_list = []
# job_description_list = []
# job_link_list = []
# job_id_list = []

In [ ]:
# driver.find_element(By.ID,"mosaic-jobResults").text
# ("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']")
# .text
# find_element_by_xpath("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']").text


# 1.) iterate over jobs, locations
# 2.) find the elements from outer to inner
# 3.) extract everything then go to next page
# 4.) medium article for each pagination
# 5.) error handling & timing, clicking

In [ ]:
# Consider a few options:

# 1.) Try to use incognito -----------(done)
# 2.) Maybe I should use random int for sleep ----(good idea)
# 3.) What to do when I have the human click button for pop up
# 4.) verify if the "+" symbols are needed look at formatting ----------(done)
# 5.) check if the formatting to parse is the same or not for div tags, etc
# 6.) do I need to use headless browser? (Possible issues! investigate)
# 7.) locating elements using the BY package, similar to beautiful soup find_all
# 8.) errors with NoSuchElementException ----------------(done)
# 9.) try to identify code that doesn't change over time
# 10.) Xpath to find buttons to go page by page and contain arrows forward/backward with try.except

# Notes for this project:

+ Filling in forms:
+ click buttons
+ possible human detection stuff

**`Xpath vs CSS selectors for retreiving data`**

+ `Xpath:` bidirectional (can go from parent to child and reverse) traversal
    + slower retrevial speed
    + text functions supported
    + pay attention to relative '//' and absolute path '/' notations
    + Think of a tree like structure to breakdown
+ `CSS:` directional (parent to child only)

`------------------------`

**`Xpath`**
+ *`Xpath`* stands for `XML Path` which is a query language used to find the path of an element in XML documents
+ Essentially you are navigating a `DOM` 
+ More flexible than using `CSS`
    + If you don't know the name of an element you can use `contains` as your key word which is great!
 
**`CSS`**
+ Most often the HTML will be styled in a cascading format and identifying elements will come from the `Class` they fall within
+ They are used to select various elements within a `DOM`
    + **`Simple selectors:`** such as finding a `Class` or `ID`
    + **`Attribute selectors:`** 
    + **`Pseudo selectors:`** such as hover boxes or check boxes as examples

In [ ]:
url="https://www.indeed.com/q-USA-jobs.html?vjk=823cd7ee3c203ac3"

In [ ]:
# function to get DOM from given URL
def get_dom(url):
    driver.get(url)
    page_content = driver.page_source
    product_soup = BeautifulSoup(page_content, 'html.parser')
    dom = et.HTML(str(product_soup))
    return dom

In [ ]:
# for i in get_dom(url):
#     print(et.tostring(i))

In [ ]:
# # functions to extract job link
# def get_job_link(job):
#     try:
#         job_link = job.xpath('./descendant::h2/a/@href')[0]
#     except Exception as e:
#         job_link = 'Not available'
#     return job_link


# # functions to extract job title
# def get_job_title(job):
#     try:
#         job_title = job.xpath('./descendant::h2/a/span/text()')[0]
#     except Exception as e:
#         job_title = 'Not available'
#     return job_title


# # functions to extract the company name
# def get_company_name(job):
#     try:
#         company_name = job.xpath('./descendant::span[@class="companyName"]/text()')[0]
#     except Exception as e:
#         company_name = 'Not available'
#     return company_name


# # functions to extract the company location
# def get_company_location(job):
#     try:
#         company_location = job.xpath('./descendant::div[@class="companyLocation"]/text()')[0]
#     except Exception as e:
#         company_location = 'Not available'
#     return company_location


# # functions to extract salary information
# def get_salary(job):
#     try:
#         salary = job.xpath('./descendant::span[@class="estimated-salary"]/span/text()')
#     except Exception as e:
#         salary = 'Not available'
#     if len(salary) == 0:
#         try:
#             salary = job.xpath('./descendant::div[@class="metadata salary-snippet-container"]/div/text()')[0]
#         except Exception as e:
#             salary = 'Not available'
#     else:
#         salary = salary[0]
#     return salary


# # functions to extract job type
# def get_job_type(job):
#     try:
#         job_type = job.xpath('./descendant::div[@class="metadata"]/div/text()')[0]
#     except Exception as e:
#         job_type = 'Not available'
#     return job_type


# # functions to extract job rating
# def get_rating(job):
#     try:
#         rating = job.xpath('./descendant::span[@class="ratingNumber"]/span/text()')[0]
#     except Exception as e:
#         rating = 'Not available'
#     return rating


# # functions to extract job description
# def get_job_desc(job):
#     try:
#         job_desc = job.xpath('./descendant::div[@class="job-snippet"]/ul/li/text()')
#     except Exception as e:
#         job_desc = ['Not available']
#     if job_desc:
#         job_desc = ",".join(job_desc)
#     else:
#         job_desc = 'Not available'
#     return job_desc

In [ ]:
# functions to extract job link
# def get_job_link(job):
#     practice_=[]
#     try:
#         job_link = job.xpath('./descendant::h2/a/@href')[0]
#     except Exception as e:
#         job_link = 'Not available'
#     practice_.append(job_link)
# #     return job_link
#     return practice_

# for job_keyword in job_search_keyword:
#     for location_keyword in location_search_keyword:
#         all_jobs = []
#         for page_no in range(0, 10, 10):
#             url = paginaton_url.format(job_keyword, location_keyword, page_no)
#             page_dom = get_dom(url)
#             jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
#         all_jobs.append(jobs)
# #             all_jobs = all_jobs + jobs
# #     print(all_jobs)


In [ ]:
all_jobs

In [ ]:
# Open a CSV file to write the job listings data
# with open('indeed_jobs1.csv', 'w', newline='', encoding='utf-8') as f:
#     theWriter = writer(f)
#     heading = ['job_link', 'job_title', 'company_name', 'company_location', 'salary', 'job_type', 'rating', 'job_description', 'searched_job', 'searched_location']
#     theWriter.writerow(heading)
#     for job_keyword in job_search_keyword:
#         for location_keyword in location_search_keyword:
#             all_jobs = []
#             for page_no in range(0, 100, 10):
#                 url = paginaton_url.format(job_keyword, location_keyword, page_no)
#                 page_dom = get_dom(url)
#                 jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
# #                 all_jobs = all_jobs +jobs
#                 print(all_jobs+jobs)
#                 all_jobs_ = all_jobs.append(jobs) #changed here and below
#                 print("yay",all_jobs_)
#             for job in all_jobs_:
#                 job_link = base_url + get_job_link(job)
#                 time.sleep(2)
#                 job_title = get_job_title(job)
#                 time.sleep(2)
#                 company_name = get_company_name(job)
#                 time.sleep(2)
#                 company_location = get_company_location(job)
#                 time.sleep(2)
#                 salary = get_salary(job)
#                 time.sleep(2)
#                 job_type = get_job_type(job)
#                 time.sleep(2)
#                 rating = get_rating(job)
#                 time.sleep(2)
#                 job_desc = get_job_desc(job)
#                 time.sleep(2)
#                 record = [job_link, job_title, company_name, company_location, salary, job_type, rating, job_desc, job_keyword, location_keyword]
#                 theWriter.writerow(record)

# # Closing the web browser
# driver.quit()

In [ ]:
for job_keyword in job_search_keyword:
    for location_keyword in location_search_keyword:
#         print(job_keyword)
        all_jobs = []
        for page_no in range(0, 10, 10): # changed 0,100,10
            url = paginaton_url.format(job_keyword, location_keyword, page_no)
            page_dom = get_dom(url)
            jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
#             all_jobs_ = all_jobs.append(jobs)
            print(jobs.text)
#             all_jobs_ = all_jobs+jobs #changed here and below
#             print("yay",all_jobs_)
#         for job in all_jobs_:
#             job_link = base_url + get_job_link(job)
#             time.sleep(2)
#             job_title = get_job_title(job)
#             time.sleep(2)
#             company_name = get_company_name(job)
#             time.sleep(2)
#             company_location = get_company_location(job)
#             time.sleep(2)
#             salary = get_salary(job)
#             time.sleep(2)
#             job_type = get_job_type(job)
#             time.sleep(2)
#             rating = get_rating(job)
#             time.sleep(2)
#             job_desc = get_job_desc(job)
#             time.sleep(2)
#             print(job_link,company_location,salary,job_type,rating)

In [ ]:
import pandas as pd

pd.read_csv('indeed_jobs1.csv')

# Like, Share & <font color=red>SUB</font>scribe

# `Citations & Help:`

# ◔̯◔

https://pypi.org/project/webdriver-manager/

https://www.blog.datahut.co/post/scrape-indeed-using-selenium-and-beautifulsoup

https://github.com/henrionantony/Dynamic-Web-Scraping-using-Python-and-Selenium/blob/master/indeed.py

https://www.specrom.com/blog/web-scraping-job-postings-on-indeed-using-python/

https://www.scrapingdog.com/blog/scrape-indeed-using-python/ (bs4 as of Feb 13, 2023)

https://selenium-python.readthedocs.io/locating-elements.html#locating-elements

https://stackoverflow.com/questions/50865088/how-to-get-string-dump-of-lxml-element

https://selenium-python.readthedocs.io/navigating.html

https://towardsdatascience.com/web-scraping-job-postings-from-indeed-com-using-selenium-5ae58d155daf (2020 version)

https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

https://medium.com/forcodesake/how-to-build-a-scraping-tool-for-indeed-in-8-minutes-data-science-csv-selenium-beautifulsoup-python-95fcca4b9719 (Good Read & Adapted Code)

https://www.tutorialspoint.com/how-to-open-browser-window-in-incognito-private-mode-using-python-selenium-webdriver

https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.keys.html

https://pythonbasics.org/selenium-wait-for-page-to-load/

https://www.seleniumeasy.com/selenium-tutorials/selenium-headless-browser-execution

https://www.browserstack.com/guide/expectedconditions-in-selenium

https://www.testim.io/blog/xpath-vs-css-selector-difference-choose/

https://www.w3.org/TR/REC-DOM-Level-1/introduction.html

https://github.com/diego-florez/Selenium-Web-Scraping/blob/master/indeed.py (Indeed scrape Selenium 2020) error Handling also

https://www.testim.io/blog/selenium-click-button/

https://scrapfly.io/blog/how-to-scrape-indeedcom/

https://goh.physics.ucdavis.edu/datascience/webscraping/webscraping.html